In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

'\nCopyright (c) Microsoft Corporation.\n'

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [14]:
api_key = os.environ["OPENAI_API_KEY"]
llm_model = "gpt-4o"
embedding_model = "text-embedding-3-small"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [15]:
# parquet files generated from indexing pipeline
INPUT_DIR = "/Users/dmitrii.koriakov/PycharmProjects/graphrag-experiments/examples/my_workflow/output/20240712-120400/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [16]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

/Users/dmitrii.koriakov/PycharmProjects/graphrag-experiments/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
/Users/dmitrii.koriakov/PycharmProjects/graphrag-experiments/graphrag/query/indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


Report records: 435


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,406,# Ford Puma ST-Line V and Related Features\n\n...,3,7.5,Ford Puma ST-Line V and Related Features,The impact severity rating is high due to the ...,The community revolves around the Ford Puma ST...,[{'explanation': 'The ST-Line V is a higher tr...,"{\n ""title"": ""Ford Puma ST-Line V and Relat...",694533a1-4279-42e7-8dfd-f3cc646ed8c3
1,407,# Ford Puma Features: Multimedia Interface and...,3,7.5,Ford Puma Features: Multimedia Interface and W...,The impact severity rating is high due to the ...,"The community revolves around the Ford Puma, s...",[{'explanation': 'The Multimedia Interface is ...,"{\n ""title"": ""Ford Puma Features: Multimedi...",1ace0aeb-ac84-48bf-af3a-c829eb60ce19
2,408,# Android Auto Integration in Various Car Mode...,3,7.5,Android Auto Integration in Various Car Models,The impact severity rating is high due to the ...,The community revolves around the Android Auto...,[{'explanation': 'Android Auto is a feature th...,"{\n ""title"": ""Android Auto Integration in V...",7e8f1ce0-97aa-455e-863e-7b2e3f90685d
3,409,# Apple CarPlay Integration in Various Car Mod...,3,7.5,Apple CarPlay Integration in Various Car Models,The impact severity rating is high due to the ...,The community revolves around the integration ...,[{'explanation': 'Apple CarPlay is widely adop...,"{\n ""title"": ""Apple CarPlay Integration in ...",fec229fd-8c69-4af8-9387-bdc310ba34a7
4,410,# 10.0-Inch Multimedia Touchscreen System and ...,3,6.5,10.0-Inch Multimedia Touchscreen System and Co...,The impact severity rating is moderate due to ...,The community revolves around the 10.0-inch mu...,[{'explanation': 'The 10.0-inch multimedia tou...,"{\n ""title"": ""10.0-Inch Multimedia Touchscr...",d564ea36-4230-4981-8775-e682d9475c3f


#### Build global context based on community reports

In [17]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [18]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [19]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [20]:
result = await search_engine.asearch(
    "What's the difference between Subaru Outback and Volkswagen Touareg in terms of engine?"
)

print(result.response)

### Engine Options and Performance

#### Volkswagen Touareg

The Volkswagen Touareg offers a diverse range of engine options, catering to various performance needs and preferences. The available engines include:

- **3.0-litre V6 TDI Diesel Engines**: These engines come in different power outputs, such as 231hp and 286hp, providing robust performance and efficiency [Data: Reports (233, 237)].
- **Plug-in Hybrid Engines**: The Touareg features advanced plug-in hybrid options, including a 381hp and a 462hp variant. The 462hp engine, particularly in the R Trim, combines a 3.0-litre combustion engine with an electric motor, delivering a total output of 462hp [Data: Reports (233, 235)].
- **3.0-litre V6 Turbocharged Engine**: The 210TDI R-Line variant is powered by a 3.0L turbocharged V6 engine, producing 210kW and 600Nm of torque, ensuring a dynamic driving experience [Data: Reports (189, 191)].
- **V8 Engine**: The Wolfsburg edition features a V8 engine that delivers 310kW of power and 90

In [21]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,376,Car Leasing and Electric Vehicles,0.361345,# Car Leasing and Electric Vehicles\n\nThe com...,7.5
1,372,Subaru XV and Ford Puma Comparison,0.235294,# Subaru XV and Ford Puma Comparison\n\nThe co...,7.5
2,167,Ford Puma ST-Line V Community,0.159664,# Ford Puma ST-Line V Community\n\nThe communi...,7.5
3,308,Audi A3 Assistance Package Features,0.067227,# Audi A3 Assistance Package Features\n\nThe c...,7.5
4,369,Subaru XV and AWD Systems,0.067227,# Subaru XV and AWD Systems\n\nThe community r...,7.5
...,...,...,...,...,...
306,203,Peugeot 5008 Engine and Transmission,0.016807,# Peugeot 5008 Engine and Transmission\n\nThe ...,4.5
307,206,5008 Crossway Interior Features,0.016807,# 5008 Crossway Interior Features\n\nThe commu...,3.0
308,45,131HP 1.5-Litre Diesel Engine and Associated S...,0.008403,# 131HP 1.5-Litre Diesel Engine and Associated...,6.5
309,28,Puma and Leather Seats Preference,0.025210,# Puma and Leather Seats Preference\n\nThe com...,4.5


In [22]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 19. LLM tokens: 218212
